# Lasagne Conv Net on Media Eval 2015 Placing Task - Locale

In [1]:
# Import Python libraries
import numpy as np
import theano
import theano.tensor as Tensor
import lasagne
import time

In [2]:
# Import own modules
import data_utils
import lasagne_model_predict_country as cnn_model

In [12]:
# hyperparameters
# hidden_size = 256 # size of hidden layer of neurons
LEARNING_RATE = 1e-2
MOMENTUM = 0.9
lr_decay = 0.995
reg_strength = 2e-2
grad_clip = 10
batchsize = 32
num_epochs = 8
dropout_p = 0.5
num_lstm_layers = 1
theano.config.optimizer='fast_compile'
theano.config.exception_verbosity='high'

In [6]:
# Load Data Set

DATA_BATCH = '000_small_'
DATA_SIZE = '48by32'
NUM_CLASSES = 5


DATA_SET = DATA_BATCH + DATA_SIZE
print 'Data Set:', DATA_SET
print 'Num classes:', NUM_CLASSES
print 'Preparing Data Set....'

X_input_filename = 'data_maps/' + DATA_SET + '/x_input.npy'
Y_output_filename = 'data_maps/' + DATA_SET + '/y_labels.npy'

X = data_utils.load_npy_file(X_input_filename)
Y = data_utils.load_npy_file(Y_output_filename)
# print 'X: {}'.format(X.shape)
# print 'Y: {}'.format(Y.shape)
# print 'Y sample ', Y[:10]

num_samples, H, W, C = X.shape

# swap C and H axes --> expected input
X = np.swapaxes(X, 1, 3)  # (num_samples, C, W, H)

#Splitting into train, val, test sets

num_train = int(num_samples * 0.8)
num_val = int(num_samples * 0.1)
num_test = num_samples - num_train - num_val

# print 'num_train: %d, num_val: %d, num_test: %d' % (num_train, num_val, num_test)

X_train = X[:num_train]
X_val = X[num_train:num_train+num_val]
X_test = X[num_train+num_val:]

y_train = Y[:num_train]
y_val = Y[num_train:num_train+num_val]
y_test = Y[num_train+num_val:]

print 'X_train', X_train.shape
print 'y_train', y_train.shape
print 'X_val', X_val.shape
print 'X_test', X_test.shape

Data Set: 000_small_48by32
Num classes: 5
Preparing Data Set....
X_train (1504, 3, 48, 32)
y_train (1504,)
X_val (188, 3, 48, 32)
X_test (188, 3, 48, 32)


In [17]:
# Prepare Theano variables for inputs and targets
input_var = Tensor.tensor4('inputs')
target_var = Tensor.ivector('targets')

# Create neural network model
cnn_model.hello_world()
model = cnn_model.build_cnn(NUM_CLASSES, inputVar = input_var)

# Create a loss expression for training, i.e., a scalar objective we want
    # to minimize (for our multi-class problem, it is the cross-entropy loss):
prediction = lasagne.layers.get_output(model)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

 # Create update expressions for training, i.e., how to modify the
# parameters at each training step. Here, we'll use Stochastic Gradient
# Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.
params = lasagne.layers.get_all_params(model, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=LEARNING_RATE, momentum=MOMENTUM)


# Create a loss expression for validation/testing. The crucial difference
# here is that we do a deterministic forward pass through the network,
# disabling dropout layers.
test_prediction = lasagne.layers.get_output(model, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()
# As a bonus, also create an expression for the classification accuracy:
test_acc = Tensor.mean(Tensor.eq(Tensor.argmax(test_prediction, axis=1), target_var),
                  dtype=theano.config.floatX)

 # Compile a function performing a training step on a mini-batch (by giving
# the updates dictionary) and returning the corresponding training loss:
train_fn = theano.function([input_var, target_var], loss, updates=updates, allow_input_downcast=True)

# Compile a second function computing the validation loss and accuracy:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

# Finally, launch the training loop.
print("Starting training...")
# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in data_utils.iterate_minibatches(X_train, y_train, 500, shuffle=True):
        inputs, targets = batch
        err = train_fn(inputs, targets)
        print("Batch {}: Train Error = {}").format(train_batches+1, err)
        train_err += err
        print("Batch {}: Total Train Error = {}").format(train_batches+1, train_err)
        train_batches += 1
            
    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in data_utils.iterate_minibatches(X_val, y_val, 500, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        print("Batch {}: Total Val Error = {}").format(val_batches+1, val_err)
        val_acc += acc
        val_batches += 1
    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

# After training, we compute and print the test error:
test_err = 0
test_acc = 0
test_batches = 0
for batch in data_utils.iterate_minibatches(X_test, y_test, 500, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs, targets)
    test_err += err
    test_acc += acc
    test_batches += 1
    
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))


hello world!
Starting training...
Batch 1: Train Error = 89.8463868279
Batch 1: Total Train Error = 89.8463868279
Batch 2: Train Error = nan
Batch 2: Total Train Error = nan
Batch 3: Train Error = nan
Batch 3: Total Train Error = nan
Epoch 1 of 8 took 32.419s
  training loss:		nan


ZeroDivisionError: integer division or modulo by zero